<a href="https://colab.research.google.com/github/surib123/Accelerating_BP_by_Using_NMF_based_methods/blob/master/Keras_Model_for_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BP+NMF_hybrid_Optimization(v2)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Basic Import ##
import numpy as np
eps=np.finfo(np.float).eps
from numpy import sqrt,zeros,ones,eye,diag,vstack,hstack,array
from numpy.linalg import svd,norm,pinv,solve
from numpy.random import permutation


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## Tensorflow ## 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

from keras.models import Sequential,Model
from keras.layers import Input,Dense,Concatenate, Flatten
from keras.layers import BatchNormalization,Dropout 
from keras.layers import Conv2D, MaxPooling2D,MaxPool2D,GlobalAveragePooling2D,AveragePooling2D
from keras.layers import ReLU,LeakyReLU

from keras.utils import to_categorical 
from keras.datasets import cifar10
from keras.optimizers import Adam,Adamax
from keras.callbacks import ModelCheckpoint

from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array # Image Related

## sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# scipy

import scipy
import scipy.misc as sm

# google colab ##
from google.colab import drive,files


## Others
import os
import time
from time import time as abtime
import copy
import itertools
#from time import time as abtime
import PIL
from termcolor import colored
from pprint import pprint
import importlib as imp

print('np.version is '+str(np.__version__))
print('pd.version is '+str(pd.__version__))
print('tf.version is '+str(tf.__version__))
print('keras.version is '+str(keras.__version__))

np.version is 1.18.5
pd.version is 1.0.5
tf.version is 2.3.0
keras.version is 2.4.0


### tic & toc

matlab-like tic&toc function

In [ ]:
def tic():
    #require to import time
    global start_time_tictoc
    start_time_tictoc = abtime()

def toc():
    if "start_time_tictoc" not in globals():
        print("tic has not been called")
    
    return abtime()-start_time_tictoc

## Basic Function

In [ ]:
def tilde(A):
    return vstack([A,ones(A.shape[1])])
    
def  appl_f(x,Type='relu'):
    if Type=='relu':
        return np.maximum(0,x)
    elif Type=='sigmoid':
        return 1/(1+np.exp(-x))
def relu(x):
    return np.maximum(0,x)

### check_br
- itr, tt : iteration and time

- X_train,X_test,Y_train,Y_test : Datasets for Check

- check_factor: {'basic','hitachi'}



In [ ]:
 def  check_br(itr,tt,X_train,Y_train,X_test,Y_test,WZ,verbose=True):

    L=len(WZ)
    WZ_tmp = compute_z_br(X_train,WZ); 
    Z = WZ_tmp[L-1]['Z']

    WZ_tmp = compute_z_br(X_test,WZ);
    Z_test = WZ_tmp[L-1]['Z'];
    
        
   ## Norm Check ###
    RR=error_norm(Y_train,Z,normtype='RRN')
    RR_each = list(norm(Y_train-Z,axis=1)/norm(Y_train,axis=1))

    RR_test=error_norm(Y_test,Z_test,normtype='RRN')
    RR_test_each = list(norm(Y_test-Z_test,axis=1)/norm(Y_test,axis=1) )
    
    ## Error check ###    
    id_Ytrain = np.argmax(Y_train,axis=0);
    id_Ztrain = np.argmax(Z,axis=0);
    id_Ytest = np.argmax(Y_test,axis=0);
    id_Ztest = np.argmax(Z_test,axis=0);
    p_train = 100-np.sum(id_Ztrain==id_Ytrain)/len(id_Ytrain)*100;
    p_test = 100-np.sum(id_Ztest==id_Ytest)/len(id_Ytest)*100; 
        
#    if check_factor=='basic':
    resvec=[itr,tt]+[RR]+[RR_test]+[p_train,p_test]
#    elif check_factor=='hitachi':
#        resvec = [itr,tt]+[RR]+RR_each+[RR_test]+RR_test_each+[p_train,p_test]
    
    
    
    if verbose==True:
        if True:
            print('{:3.3f}  {:6.3f}  {:12.3f} {:12.3f}  {:12.3f}  {:12.3f} \n'.format(itr,tt,RR,RR_test,p_train,p_test))
  #      elif check_factor=='hitachi':
  #          print('{:3.3f}  {:6.3f}  ---☉  [{:6.3f}] {:6.3f}  {:6.3f}  {:6.3f} {:6.3f} ---  ♦︎[{:6.3f}] {:6.3f} {:6.3f}  {:6.3f}  {:6.3f} \n'.format(itr,tt,RR,*RR_each,RR_test,*RR_test_each));        

    return resvec

### error_norm
[arg]

data :<br>
$X =[x_1,x_2,...,x_m]^T$<br>
$Y =[y_1,y_2,...,y_m]^T$<br>
normtype: the type of norm$\in( ${'RRN','MSE''})

[arg]
S: the scaled data

In [ ]:
def error_norm(X,Y,normtype='RRN'):
    if normtype=='RRN':
        err=norm(X-Y,'fro')/norm(X,'fro')
    if normtype=='MSE':
        err=norm(X-Y,'fro')/X.shape[0]
    return err

### weight_formatter

In [ ]:
def weight_formatter(weights,Type='nmf_shape'):
    

# Evaluate  the type of input Weights
   
    if type(weights)==dict: # check wheteher it has WZ shape
        input_type='WZ' 
        
    else: 
        try: # when not it means nmf shape or keras shape
            weights[1]# check if it has second value 
        except: # except means Wb has nmf_shape
            input_type='nmf_shape'
        else:
            if len(weights[1].shape) !=1: # if second elements is vector ( it means bias )
                input_type='nmf_shape'
            else:
                input_type='keras_shape'
    
    
## Formmat the type of Input
    new_weights=[]
    
    # the case of Type =='nmf shape'
    if Type=='nmf_shape':
        if input_type=='nmf_shape':
            new_weights=weights
        elif input_type=='keras_shape':
            L=int(len(weights)/2)
            for i in range(L):
                new_weights.append(vstack([weights[2*i],weights[2*i+1]]).T )   # get 
        elif input_type=='WZ':
            L=int(len(weights))
            for i in range(L):
                new_weights.append(weights[i]['W'] )
                
    if Type=='WZ':
        if input_type=='nmf_shape':
            L=int(len(weights))
            for i in range(L):
                new_weights.append(WZ_object(W=weights[i]) )
        elif input_type=='keras_shape':
            L=int(len(weights)/2)
            for i in range(L):
                new_weights.append(WZ_object(W=vstack([weights[2*i],weights[2*i+1]]).T) )
        elif input_type=='WZ':
            new_weights=weights
            
    if Type=='keras_shape':
        if input_type=='nmf_shape':
            new_weights=[]
            for l in range (len(weights)):
                new_weights.append(weights[l][:,:-1].T)
                new_weights.append(weights[l][:,-1])
        elif input_type=='WZ':
            new_weights=[]
            for l in range(len(weights)):
                new_weights.append(weights[l].W[:,:-1].T)
                new_weights.append(weights[l.W][:,-1])
            
                
    
    
    return new_weights

### low_rank_appl

In [ ]:
def low_rank_appl(A,delta):
    U,s,V=np.linalg.svd(A,full_matrices=False)

      # get maximum of singular values and normalize it to be 1
    k=np.sum(s/s[0]>delta)
    Asvd=dict([('U',U[:,0:k]),('S',np.diag(s[0:k])),('V',V[0:k,:].T)])
    
    return Asvd

In [ ]:
low_rank_appl(np.random.randn(5,5),delta=0.1).keys()

dict_keys(['U', 'S', 'V'])

In [ ]:
def compute_z_br(X,WZ): # get all activations of FC-Layer
    L=len(WZ)
    WZ[0]['Z'] = appl_f(WZ[0]['W']@tilde(X));                                            # Forward Prop of Inpur layer

    for i in range(1,L-1):                                                                 # Forward Prop of  Middle layer

            WZ[i]['Z'] = appl_f(WZ[i]['W']@tilde(WZ[i-1]['Z']));

    WZ[L-1]['Z'] = WZ[L-1]['W'] @tilde(WZ[L-2]['Z']);                                   #   Forward Prop of  Output layer

    return WZ

In [ ]:
def nmf_br(Type,A,U,V,alpha,beta,iter1,iter2,iter3,delta,act_func='relu'):
    
    n=V.shape[1];
    Va=vstack([V,ones(V.shape[1])]);
    IE=ones([n,n])
    
    
    if Type == 'N' :# Normal NMF: min_U,V ||A - U V|| s.t. U, V >= 0
    
        for i  in range(iter1) :
            U_org=U[:,:-1]
            
            V = V * (U_org.T@A) / ((U_org.T@U)@Va+beta*V+eps)
            Va=vstack([V,ones(V.shape[1])])
            U = U * (A@Va.T) / (U@(Va@Va.T)+alpha*U+eps)
            
            
            
    elif Type == 'S': # Semi-NMF: min_U,V ||A - U V|| s.t. V >= 0
    
        for i   in range(iter1):
            
            Vsvd = low_rank_appl(Va,delta)
            R=A-U@Va
            U = U + (((R @ Vsvd['V']) / diag(Vsvd['S'])) @ Vsvd['U'].T)
            ss=diag(Vsvd['S']);ss=ss**2/(alpha+ss**2)
    
                    
                    
            U = U@(( Vsvd['U']*ss) @ Vsvd['U'].T);        # U = A / V
            UtA = U[:,:-1].T@A;
            UtAp = (abs(UtA) + UtA) / 2;
            UtAm = (abs(UtA) - UtA) / 2;
            UtU =  U[:,:-1].T@U;
            UtUp = (abs(UtU) + UtU) / 2;
            UtUm = (abs(UtU) - UtU) / 2;

            V = V * sqrt( (UtAp + UtUm@Va + beta*V) / (UtAm + UtUp@Va+beta*V + eps) )
            Va=vstack([V,ones(V.shape[1])])
    elif Type == 'NS': # Nonlinear Semi-NMF: min_U,V ||A - f(U V)|| s.t. V >= 0
    
        for  i  in range(iter1):
            usv = low_rank_appl(Va,delta)
            U = non_linear_lsq_br('XA',V,usv,A,U,alpha,iter2,act_func)
    
            usv = low_rank_appl(U[:,:-1],delta)
            V = non_linear_lsq_br('AX',U,usv,A,V,beta,iter3,act_func)
            

    
    return U,V

In [ ]:
def non_linear_lsq_br(AXorXA,A,Asvd,B,X,lamb,itermax,act_func='relu'):
    U = Asvd['U']
    S = Asvd['S']
    V = Asvd['V']
    omega = 1.0
        
    ##### Normal Mode ######
    if AXorXA == 'AX': # min_X || B - f(A X) ||
        
        Xa = np.vstack([X,ones(X.shape[1])])
        usv = low_rank_appl(A[:,:-1].T@A[:,:-1],1e-14);
      
            
        for k in range(itermax):
            R = B - appl_f(A@Xa,act_func);
            X = X + omega * (V @( (1/diag(S))[:,np.newaxis]* (U.T @ R)));
            X = solve((eye(A.shape[1]-1)+lamb*(usv['V'] @ ( (1/diag(usv['S']))[:,np.newaxis]*usv['U'].T))),X) #Regularziation Term
            X = relu(X) # non-negative constraint

    elif AXorXA == 'XA': # min_X || B - f(X A) ||
        Aa=A
        for k in range(itermax):
            Aa = np.vstack([A,ones(A.shape[1])])
            R = B - appl_f(X@Aa,act_func)
            X = X + omega * (((R @ V) /np.diag( S)) @ U.T) #    X<- X+ w  R A♱  
            ss = diag(S);ss = ss**2/(lamb+ss**2);X = X@((U*ss)@U.T)
           
                
    return X
        

## Add Customized Class for keras model




In [ ]:
####### 1 #######
inputs = Input(shape=(784)) # Input
####### 2  #######
x = Dense(300, activation="relu")(inputs)
x = Dense(150, activation="relu")(x)
x = Dense(10, activation="linear")(x)

####### 3 #######
model = Model(inputs, x)

In [ ]:
class MyModel(Model):
########################################
    def __init__(self,inputs,outputs):
        super(MyModel,self).__init__(inputs,outputs) #
        
        self.initializer='randn'
        self.delta=[0,0]
        self.aeitr=[10,5,5]
        self.ftitr=[10,1,1]
        self.nsnmf=[10,1,1]
        self.nmf_batch=[0,500]
        self.nmf_lamb=[1e-3,1e-3]

        self.optimizer=Adam()
        self.bp_epochs=10
        self.bp_batch=[50,50]
        self.bp_lamb=[1e-3,1e-3]

        self.init_time=time.time()
        self.cur_epoch=0 # current epoch
        self.record=[] # 

        self.verbose=True
        
        
        self.compile(loss='mean_squared_error',optimizer='adam',metrics='accuracy')
        #self.WZ_weights=self.get_WZ_weights()
        self.num_dense=self.get_num_denses()


        def get_epoch(self,epoch):  
            if self.dnn_solver=='nmf':
                self.ftitr[0]=epoch
            elif self.dnn_solver=='bp':
                    self.bp_epochs=epoch
    
    def reset_record(self):
        self.record=[]
        self.init_time=time.time()
        
    def check_record(self,X_train,Y_train,X_test,Y_test,verbose=True): # method for train record
        self.cur_epoch+=1
        # get losses and errors of Train & Test
        trainloss,trainerror=tuple(self.evaluate(X_train,Y_train,verbose=False))
        testloss,testerror=tuple(self.evaluate(X_test,Y_test,verbose=False))

        self.record.append([self.cur_epoch,time.time()-self.init_time,trainloss,testloss,trainerror,testerror])

        if verbose:
            print('{:3.3f}  {:6.3f}  {:12.3f} {:12.3f}  {:12.3f}  {:12.3f} \n'.format(self.cur_epoch,time.time()-self.init_time,trainloss,testloss,trainerror,testerror))

    def get_num_denses(self):
        num_dense=0
        for layertype in map(type,self.layers):
          if layertype == Dense:
            num_dense+=1
        return num_dense
#------------------------------#------------------------------#------------------------------#------------------------------
    def get_WZ_weights(self):       # method of transform [[W1],[b1],[W2],[b2],,,] like weights  to [{'W':[W1,b1],'Z':Z},....] weights
        results=list()
        num_dense=self.num_dense                  
        weights=self.get_weights()[-2*num_dense:]    # get all kernels(W) and bias(b) of Dense Layer

        for i in range(num_dense): # transform weights
            results.append({'W':vstack([weights[2*i],weights[2*i+1]]).T,'Z':None})
        return results

#------------------------------#------------------------------#------------------------------#------------------------------
    def get_weights_from_WZ(self):   # method that enables the class set weights from WZ_weights
        All_list=list(map(lambda x : [x['W'][:,:-1].T,x['W'][:,-1]],self.WZ_weights))    #connect all W and b
        return sum(All_list,[])
        

##########################################################################################
    def fitwith_NMF(self,X_train,Y_train,X_test,Y_test): # NMF Training
        #====== Get Default Param=====================
        #self.reset_record()
        m,nx=X_train.shape
        _,ny=Y_train.shape
        delta = self.delta; 
        L=len(self.layers)-1 # the number of weights "-1 is because of input layer"
        # aeitr = self.aeitr;
        ftitr = self.ftitr;
        nsnmf = self.nsnmf;
        batch = self.nmf_batch; 
        if (batch[1]=='full'): batch[1]=m;# full
        lamb=self.nmf_lamb;
        verbose=self.verbose

        resvec=list()
        X_train,Y_train,X_test,Y_test=X_train.T,Y_train.T,X_test.T,Y_test.T
        
        WZ=self.WZ_weights
        Xusv = low_rank_appl(tilde(X_train),delta[0]) 
        # ========== NMF-Based Learning =====================
        print('Iter          sec           TRN              TSN               TRE              TSE           ',colored( 'NMF','red'),'\n')
        for itr in range(ftitr[0]):
            ind = np.reshape(permutation(m),[batch[1],int(m/batch[1])])
            for kk in range(int(m/batch[1])):
                
                
                XI = X_train[:,ind[:,kk]]
                YI = Y_train[:,ind[:,kk]]

                XIusv=copy.deepcopy(Xusv); XIusv['V']=Xusv['V'][ind[:,kk],:] # SVD value of Batch

                WZ = compute_z_br(XI,WZ);
                WZ[L-1]['W'],WZ[L-2]['Z']= nmf_br('S',YI,WZ[L-1]['W'],WZ[L-2]['Z'],lamb[0],lamb[1],ftitr[1],0,0,0);


                for i in range( L-2,0,-1):
                    WZ[i]['W'],WZ[i-1]['Z'] = nmf_br('NS',WZ[i]['Z'],WZ[i]['W'],WZ[i-1]['Z'],lamb[0],lamb[1],ftitr[2],nsnmf[0],nsnmf[1],delta[1]);   
                WZ[0]['W'] = non_linear_lsq_br('XA',XI,XIusv,WZ[0]['Z'],WZ[0]['W'],lamb[0],nsnmf[0]);
            
        #============   Check Performance ==========================
            self.WZ_weights=WZ
            self.set_weights(self.get_weights_from_WZ())
            self.check_record(X_train.T,Y_train.T,X_test.T,Y_test.T)
            #resvec.append(check_br(stnum[0]+itr+1,stnum[1]+abtime()-t0,X_train,Y_train,X_test,Y_test,WZ,verbose=verbose))

            
       
    def fitwith_BP(self,X_train,Y_train,X_test,Y_test): # NMF Training
            #====== Get Default Param=====================
            #self.reset_record()
            m,nx=X_train.shape
            _,ny=Y_train.shape
            delta = self.delta; 
            L=len(self.layers)-1 # the number of weights "-1 is because of input layer"
            # aeitr = self.aeitr;
            ftitr = self.ftitr;
            nsnmf = self.nsnmf;
            batch = self.bp_batch; 
            epoch=self.bp_epochs;
            lamb=self.nmf_lamb;

            for i in range(epoch):
                self.fit(X_train,Y_train,batch_size=batch[1],verbose=0)
                self.check_record(X_train,Y_train,X_test,Y_test)
##########################################################################################
#mymodel = MyModel(inputs, x)


ValueError: ignored

In [ ]:
pprint(mymodel.record)

ListWrapper([])


In [ ]:
mymodel.compile(optimizer='rmsprop',loss='mse',metrics='mse')

In [ ]:
def my_metric_fn(y_true, y_pred):
    return tf.norm(y_true-y_pred)/tf.norm(y_true)  # Note the `axis=-1`

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[my_metric_fn])

In [ ]:
mymodel.fit(X_train,Y_train,epochs=3)

Epoch 1/3


ValueError: ignored

# CNN

In [ ]:
from keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [ ]:
X_train=np.expand_dims(X_train,axis=-1)

In [ ]:

input = Input(shape=(28,28,1)) # you must set 1 
activation=ReLU()
initializer=keras.initializers.random_normal()

x = Conv2D(filters=64,kernel_size=(3,3), activation=activation,padding='same',kernel_initializer=initializer)(input)

x = Conv2D(filters=64,kernel_size=(3,3), activation=activation,padding='same',kernel_initializer=initializer)(x)


x = Conv2D(filters=32,kernel_size=(3,3), activation=activation,padding='same',kernel_initializer=initializer)(x)

x = Conv2D(filters=32,kernel_size=(3,3), activation=activation,padding='same',kernel_initializer=initializer)(x)

x = AveragePooling2D()(x)

x= Flatten()(x)

x= Dense(300)(x)

x= Dense(10)(x)

model = MyModel(input, x)

In [ ]:
model.get_WZ_weights()

[{'W': array([[-0.00203366, -0.02025667,  0.0202229 , ..., -0.01082826,
           0.01224338,  0.        ],
         [ 0.01361105,  0.02339366,  0.00156727, ..., -0.01318271,
          -0.01872476,  0.        ],
         [ 0.02019166, -0.01369226, -0.00999068, ...,  0.00746771,
           0.02910009,  0.        ],
         ...,
         [ 0.0174809 , -0.02601299, -0.01999984, ..., -0.01785457,
           0.0243574 ,  0.        ],
         [ 0.0028744 , -0.01640949,  0.02266458, ...,  0.01145671,
          -0.02541656,  0.        ],
         [ 0.00770394,  0.02043208,  0.02519836, ...,  0.01589269,
           0.01580513,  0.        ]], dtype=float32), 'Z': None},
 {'W': array([[ 0.00094382,  0.11984356, -0.11812676, ...,  0.02253154,
          -0.06001581,  0.        ],
         [-0.07394361, -0.10276604,  0.06581879, ...,  0.08337736,
           0.05033594,  0.        ],
         [-0.12549125,  0.03433639,  0.12487899, ..., -0.04949808,
           0.11484776,  0.        ],
         ..

In [ ]:
type(model.layers[-1])==Dense

True

In [ ]:
type(model.layers[-3])==Dense

False

In [ ]:

print(num_dense)

2


<bound method Person.name of <__main__.Person object at 0x7fe34adfa2e8>>
park
